In [36]:
import hickle as hkl
import numpy as np

array_hkl = hkl.load('data/train_mat.hkl')

/opt/anaconda3/lib/python3.8/site-packages/hickle/hickle.py:556: UserWarning: Input argument 'file_obj' appears to be a file made with hickle v3. Using legacy load...
  warnings.warn("Input argument 'file_obj' appears to be a file made"


In [37]:
array_hkl


array([[False, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       [False, False, False, ...,  True, False, False],
       ...,
       [False, False, False, ...,  True,  True, False],
       [False,  True,  True, ...,  True, False, False],
       [False,  True, False, ..., False, False, False]])

In [38]:
array_hkl.shape

(3200, 800)

In [4]:
import pandas as pd

s2_df = pd.read_csv('//Users/slevine-gottreich/github/DeepSilencer/data/s2.csv')
# /Users/slevine-gottreich/github/DNABERT/data/jayavulu/regression/dev.tsv')

s2_df

,Unnamed: 0,chr,start,end,log2FC,seq
0,921,chr10,32837196,32837396,2.057311,gctattttaaagagagcaggagtatgaactggaggggcagagtcag...
1,7124,chr8,101981456,101981656,1.900275,taaagtctgtgatctttctgataagtgcatagaagaaaacgctgac...
2,3151,chr18,48755796,48755996,1.532728,attccacttccatttgatgatgaaatactgctgtgtaggcccaact...
3,249,chr1,57998096,57998296,1.525627,ctggcatttttcacactatactctaatttggtttatctttcccatt...
4,1201,chr11,13913576,13913776,1.510588,taaggtctataagagagagcacctgcctagttttgcaaagtcacac...
...,...,...,...,...,...,...
1887,1201,chr11,13913576,13913776,1.510588,taaggtctataagagagagcacctgcctagttttgcaaagtcacac...
1888,249,chr1,57998096,57998296,1.525627,ctggcatttttcacactatactctaatttggtttatctttcccatt...
1889,3151,chr18,48755796,48755996,1.532728,attccacttccatttgatgatgaaatactgctgtgtaggcccaact...
1890,7124,chr8,101981456,101981656,1.900275,taaagtctgtgatctttctgataagtgcatagaagaaaacgctgac...


In [18]:
from tqdm import tqdm_notebook, tqdm
import numpy as np
acgt2num = {'A': 0,'C': 1,'G': 2,'T': 3}

def seq2mat(seq):
    seq = seq.upper()
    h = 4
    w = len(seq)
    mat = np.zeros((h, w), dtype=bool)  # True or false in mat
    for i in range(w):
        mat[acgt2num[seq[i]], i] = 1.
    return mat.reshape((1, -1))


In [19]:
tqdm.pandas()
s2_df['mat'] = s2_df.progress_apply(lambda x: seq2mat(x['seq']),axis=1)
s2_df

100%|██████████| 1892/1892 [00:00<00:00, 17195.47it/s]


,Unnamed: 0,chr,start,end,log2FC,seq,mat
0,921,chr10,32837196,32837396,2.057311,gctattttaaagagagcaggagtatgaactggaggggcagagtcag...,"[[False, False, False, True, False, False, Fal..."
1,7124,chr8,101981456,101981656,1.900275,taaagtctgtgatctttctgataagtgcatagaagaaaacgctgac...,"[[False, True, True, True, False, False, False..."
2,3151,chr18,48755796,48755996,1.532728,attccacttccatttgatgatgaaatactgctgtgtaggcccaact...,"[[True, False, False, False, False, True, Fals..."
3,249,chr1,57998096,57998296,1.525627,ctggcatttttcacactatactctaatttggtttatctttcccatt...,"[[False, False, False, False, False, True, Fal..."
4,1201,chr11,13913576,13913776,1.510588,taaggtctataagagagagcacctgcctagttttgcaaagtcacac...,"[[False, True, True, False, False, False, Fals..."
...,...,...,...,...,...,...,...
1887,1201,chr11,13913576,13913776,1.510588,taaggtctataagagagagcacctgcctagttttgcaaagtcacac...,"[[False, True, True, False, False, False, Fals..."
1888,249,chr1,57998096,57998296,1.525627,ctggcatttttcacactatactctaatttggtttatctttcccatt...,"[[False, False, False, False, False, True, Fal..."
1889,3151,chr18,48755796,48755996,1.532728,attccacttccatttgatgatgaaatactgctgtgtaggcccaact...,"[[True, False, False, False, False, True, Fals..."
1890,7124,chr8,101981456,101981656,1.900275,taaagtctgtgatctttctgataagtgcatagaagaaaacgctgac...,"[[False, True, True, True, False, False, False..."


In [35]:
np.vstack(s2_df.mat).shape

(1892, 800)

In [41]:
new_data = np.vstack(s2_df.mat)
from sklearn.model_selection import train_test_split

train,test = train_test_split(
    new_data, test_size=0.33, random_state=42)


In [45]:
import hickle as hkl
hkl.dump(train, 'data/train.hkl', mode='w')
hkl.dump(train, 'data/test.hkl', mode='w')
